In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import sys

sys.path.append('./helpers_models/')
sys.path.append('./data_visualization_and_augmentations/')
sys.path.append('../torch_videovision/')
sys.path.append('./important_csvs/')

from helpers_resnet import *

resnet = torchvision.models.resnet50(pretrained=True)
adaptive_pooling = AdaptiveConcatPool2d()
head = Head()
resnet.avgpool = adaptive_pooling
resnet.fc = head

os.environ['CUDA_VISIBLE_DEVICES']='0,1,2'

resnet = resnet.cuda()

for param in resnet.parameters():
    param.requires_grad = False
    
for param in resnet.avgpool.parameters():
    param.requires_grad = True
    
for param in resnet.fc.parameters():
    param.requires_grad = True

resnet = nn.DataParallel(resnet)
check_freeze(resnet.module)

conv1 False
bn1 False
relu True
maxpool True
layer1 False
layer2 False
layer3 False
layer4 False
avgpool True
fc True


In [3]:
from helpers_thresholds import *

In [4]:
plt.rcParams['figure.figsize'] = (12,6)
font = {'family' : 'DejaVu Sans',  'weight' : 'normal',  'size'  : 40}
plt.rc('font', **font)

In [5]:
tensor_transform = get_tensor_transform('ImageNet', False)
train_spat_transform = get_spatial_transform(2)
train_temp_transform = get_temporal_transform()
valid_spat_transform = get_spatial_transform(0)
valid_temp_transform = va.TemporalFit(size=16)

bs = 50
root_dir = '/media/scratch/astamoulakatos/nsea_video_jpegs/'
df = pd.read_csv('./small_dataset_csvs/events_with_number_of_frames_stratified.csv')
# df_train = get_df(df, 20, True, False, False)
# class_image_paths, end_idx, idx_label= get_indices(df_train, root_dir)
# train_loader = get_loader(1, 128, end_idx, class_image_paths, train_temp_transform, train_spat_transform, tensor_transform, False, True, True, 1)
df_valid = get_df(df, 20, False, True, False)
class_image_paths, end_idx, idx_label = get_indices(df_valid, root_dir)
valid_loader, valid_dataset = get_loader(1, bs, end_idx, class_image_paths, valid_temp_transform, valid_spat_transform, tensor_transform, False, True, True, 1)
df_test = get_df(df, 20, False, False, True)
class_image_paths, end_idx, idx_label = get_indices(df_test, root_dir)
test_loader, test_dataset = get_loader(1, bs, end_idx, class_image_paths, valid_temp_transform, valid_spat_transform, tensor_transform, False, True, True, 1)

torch.cuda.empty_cache()

In [6]:
load = True
if load:
    checkpoint = torch.load('/media/scratch/astamoulakatos/saved-resnet-models/fifth-small-aug-final/best-checkpoint-010epoch.pth')
    resnet.load_state_dict(checkpoint['model_state_dict'])
    print('loading pretrained freezed model!')

loading pretrained freezed model!


In [7]:
device = torch.device('cuda')

In [8]:
from predictions import *

In [9]:
resnet

DataParallel(
  (module): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Sequential(
      

In [10]:
from skorch.net import NeuralNet

In [11]:
net = NeuralNet(
    resnet.module,
    max_epochs=20,
    verbose=0,
    criterion=nn.BCEWithLogitsLoss(),
    device='cuda',
    iterator_train__shuffle=True,
    iterator_train__num_workers=4,
)

In [12]:
net

<class 'skorch.net.NeuralNet'>[uninitialized](
  module=ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (d

In [13]:
valid_dataset[0]

(tensor([[[-1.9124, -1.9295, -1.9124,  ..., -1.8610, -1.8097, -1.8439],
          [-1.8439, -1.8610, -1.8268,  ..., -1.8610, -1.8610, -1.8268],
          [-1.7754, -1.8097, -1.7412,  ..., -1.9295, -1.9295, -1.9809],
          ...,
          [-1.6555, -1.6042, -1.5185,  ..., -1.5870, -1.6384, -1.5699],
          [-2.0323, -2.0152, -1.8953,  ..., -2.0494, -2.0494, -2.0837],
          [-1.9980, -1.9638, -1.9809,  ..., -2.1179, -2.0494, -2.0494]],
 
         [[-1.8256, -1.8431, -1.8606,  ..., -1.7731, -1.7206, -1.7556],
          [-1.7556, -1.7731, -1.7731,  ..., -1.7731, -1.7731, -1.7381],
          [-1.7206, -1.7556, -1.7381,  ..., -1.8431, -1.8256, -1.8782],
          ...,
          [-1.4755, -1.4230, -1.3354,  ..., -1.2304, -1.2304, -1.1604],
          [-1.8606, -1.8431, -1.7206,  ..., -1.6506, -1.6155, -1.6506],
          [-1.8256, -1.7906, -1.8081,  ..., -1.7206, -1.6155, -1.6155]],
 
         [[-1.5953, -1.6127, -1.6127,  ..., -1.6302, -1.5779, -1.6127],
          [-1.5256, -1.5430,

In [14]:
net.fit(valid_dataset, y=None);

TypeError: forward() missing 2 required positional arguments: 'input' and 'target'

In [23]:
from skmultilearn.problem_transform import BinaryRelevance

In [24]:
clf = BinaryRelevance(classifier=net, require_dense=[True,True])

In [25]:
clf

BinaryRelevance(classifier=<class 'skorch.net.NeuralNet'>[uninitialized](
  module=ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (l...
      (headbn1): BatchNorm1d(4096, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (headdr1): Dropout(p=0.25, inplace=False)
      (fc1): Linear(in_features=4096, out_features=512, bias=True)
      (headre1): ReLU(inplace=True)
      (headbn2): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (headdr2): Dropout(p=0.5, inplace=False)
      (fc2): Linear(in_features=512, out_features=5, bias=True)
    )
  ),
),
                require_dense=[True, True])

In [26]:
clf.fit(valid_loader.dataset)

TypeError: fit() missing 1 required positional argument: 'y'

In [45]:
y_train

array([[0., 1., 0., 0., 0.],
       [0., 1., 0., 0., 0.],
       [0., 1., 0., 0., 0.],
       ...,
       [1., 0., 0., 1., 0.],
       [1., 0., 0., 1., 0.],
       [1., 0., 0., 1., 0.]], dtype=float32)

In [46]:
labels = [y for x, y in iter(valid_loader.dataset)]

In [47]:
labels

[tensor([0., 1., 0., 0., 0.]),
 tensor([0., 1., 0., 0., 0.]),
 tensor([0., 1., 0., 0., 0.]),
 tensor([0., 1., 0., 0., 0.]),
 tensor([0., 1., 0., 0., 0.]),
 tensor([0., 1., 0., 0., 0.]),
 tensor([0., 1., 0., 0., 0.]),
 tensor([0., 1., 0., 0., 0.]),
 tensor([0., 1., 0., 0., 0.]),
 tensor([0., 1., 0., 0., 0.]),
 tensor([0., 1., 0., 0., 0.]),
 tensor([0., 1., 0., 0., 0.]),
 tensor([0., 1., 0., 0., 0.]),
 tensor([0., 1., 0., 0., 0.]),
 tensor([0., 1., 0., 0., 0.]),
 tensor([0., 1., 0., 0., 0.]),
 tensor([0., 1., 0., 0., 0.]),
 tensor([0., 1., 0., 0., 0.]),
 tensor([0., 1., 0., 0., 0.]),
 tensor([0., 1., 0., 0., 0.]),
 tensor([0., 1., 0., 0., 0.]),
 tensor([0., 1., 0., 0., 0.]),
 tensor([0., 1., 0., 0., 0.]),
 tensor([0., 1., 0., 0., 0.]),
 tensor([0., 1., 0., 0., 0.]),
 tensor([0., 1., 0., 0., 0.]),
 tensor([0., 1., 0., 0., 0.]),
 tensor([0., 1., 0., 0., 0.]),
 tensor([0., 1., 0., 0., 0.]),
 tensor([0., 1., 0., 0., 0.]),
 tensor([0., 1., 0., 0., 0.]),
 tensor([0., 1., 0., 0., 0.]),
 tensor(

In [48]:
torch.stack(labels)

tensor([[0., 1., 0., 0., 0.],
        [0., 1., 0., 0., 0.],
        [0., 1., 0., 0., 0.],
        ...,
        [1., 0., 0., 1., 0.],
        [1., 0., 0., 1., 0.],
        [1., 0., 0., 1., 0.]])

In [49]:
y_train = np.array(torch.stack(labels))

In [50]:
x_train = valid_loader.dataset

In [75]:
clf.fit(images, labels)

ValueError: unrecognized csr_matrix constructor usage

In [52]:
len(y_train)

6567

In [55]:
len(x_train)

6481

In [58]:
len(valid_loader.dataset)

6481

In [70]:
labels = [y for x, y in iter(valid_loader.dataset)]

In [71]:
images = [x for x, y in iter(valid_loader.dataset)]

In [60]:
labels

[tensor([[1., 0., 0., 1., 0.],
         [1., 0., 0., 0., 0.],
         [1., 0., 1., 0., 0.],
         [0., 1., 0., 0., 0.],
         [0., 1., 0., 0., 0.],
         [1., 0., 0., 0., 1.],
         [0., 1., 0., 0., 0.],
         [1., 0., 0., 0., 1.],
         [1., 0., 0., 0., 1.],
         [1., 0., 1., 0., 0.],
         [1., 0., 1., 0., 0.],
         [0., 1., 0., 0., 0.],
         [1., 0., 0., 0., 0.],
         [0., 1., 0., 0., 0.],
         [1., 0., 0., 0., 1.],
         [1., 0., 0., 0., 0.],
         [0., 1., 0., 0., 0.],
         [1., 0., 1., 0., 0.],
         [1., 0., 0., 1., 0.],
         [0., 1., 0., 0., 0.],
         [1., 0., 0., 0., 1.],
         [0., 1., 0., 0., 0.],
         [1., 0., 0., 0., 1.],
         [1., 0., 0., 0., 1.],
         [1., 0., 0., 1., 0.],
         [1., 0., 0., 0., 1.],
         [1., 0., 0., 0., 1.],
         [1., 0., 0., 0., 0.],
         [1., 0., 0., 0., 1.],
         [0., 1., 0., 0., 0.],
         [1., 0., 0., 1., 0.],
         [1., 0., 0., 0., 1.],
        

In [66]:
y_train = np.array(torch.cat(labels))

In [67]:
len(y_train)

6450

In [ ]:
images = 

In [73]:
len(labels), len(images)

(6567, 6567)

In [ ]:
torch.stack(labels)

In [76]:
type(valid_loader.dataset)

new_dataloader.MyDataset